# Load Modules

In [47]:
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt 
from IPython.display import display
from tqdm import tqdm

# Load Source list files

In [62]:
src_list = pd.read_csv('../source_list/cv_segmented_list.txt' , header=None , names = ['fname'] , na_values='NaN')['fname'].to_list()
src_list
source = pd.DataFrame()
for s in src_list[:]:
    temp = pd.read_csv('../source_list/cv_updated/good/'+s).iloc[:,1:]
    source = source.append(temp)
source = source[source['num_obs']>0]
source = source.replace(np.nan , np.nan).sort_values('num_obs' , ascending=False).reset_index(drop=True)
cv_class = ['N', 'Candidate_CV', 'CataclyV', 'Candidate_Nova', 'Candidate',
       'DN', 'known_CV', 'Cataclysmic_Variable', 'Known_CV']
cv_class = [
    'Candidate_CV', 'CataclyV', 'Candidate_Nova', 'Candidate',
       'DN', 'known_CV', 'Cataclysmic_Variable', 'Known_CV'
] 
source = source[source['Type'].isin(cv_class)].reset_index(drop=True)
display(source)

,Name,Disc. Date,Mag.,R.A.,Dec.,num_obs,Type
0,CSS120424:125906+242634,2012/04/24,NaN,12:59:05.78,+24:26:34.4,572,Candidate_CV
1,W32,NaN,NaN,00:24:05.716,-72:04:49.44,556,CataclyV
2,CXOGlb_J002405.2-720446,NaN,NaN,00:24:05.216,-72:04:46.59,523,CataclyV
3,Cl__NGC_104_MAM_M54898,NaN,NaN,00:24:06.013,-72:04:56.20,523,CataclyV
4,CXOGlb_J002404.9-720455,NaN,NaN,00:24:04.913,-72:04:55.36,478,CataclyV
...,...,...,...,...,...,...,...
309,N2403_2008-01a,NaN,NaN,07:37:01.33,+65:35:14.7,1,Candidate_Nova
310,MASTER_OT_JJ040556.85+714813.1,2013/Dec/23,18.2,04:05:56.85,+71:48:13.1,1,Known_CV
311,CX25,NaN,NaN,19:10:51.957,-59:58:40.55,1,Candidate_CV
312,MASTER_OT_JJ105545.20+573109.7,2014/Mar/04,15.5,10:55:45.20,+57:31:09.7,1,Known_CV


In [63]:
source['Type'].unique()

array(['Candidate_CV', 'CataclyV', 'Candidate_Nova', 'Candidate', 'DN',
       'known_CV', 'Cataclysmic_Variable', 'Known_CV'], dtype=object)

# Read Chandra data files 

In [64]:
ch_names = []
ch_ra = []
ch_dec = []
offset = []
src_id_list = [] 
n_obs_list = []
df_combined = pd.DataFrame()
for index in tqdm(range(len(source))):
    s = source['Name'][index]
    df = pd.read_csv('../data/all_data/CV_v2_all/'+s+'.csv' , index_col='index').sort_values('sepn').reset_index(drop=True)
    #display(df)
    top_match = df.loc[0]
    # Select only the source , which has highest cross-matching 
    # And create an array with this seperation value to be stored in original source list table
    # such that source list table have chandra source name , number of actual obs  and seperation.
    # Select all the obs of this cross match and append in all data file

    # Put actual source name , and give source id and obs id in this data
    df_select = df[df['name']==top_match['name']].reset_index(drop=True)
    n_obs_cur = len(df_select)
    obs_id = ['CV_'+str(index)+'_obs_'+str(i) for i in range(len(df_select))]
    src_id = ['CV_'+str(index)]*len(df_select)
    df_select.insert(0 ,'src_id' , src_id)
    df_select.insert(1 ,'obs_id' , obs_id)
    df_select.insert(2, 'num_obs' , [str(len(df_select))]*len(df_select))
    top_row = df_select.iloc[0]
    ch_names.append(top_row['name'])
    ch_ra.append(top_row['ra'])
    ch_dec.append(top_row['dec'])
    offset.append(top_row['sepn'])
    n_obs_list.append(len(df_select))
    src_id_list.append('CV_'+str(index))
    #display(df_select)
    df_combined = df_combined.append(df_select)
source_updated = source.copy().reset_index(drop=True)
source_updated.insert(7 , 'chandra_name' , ch_names)
source_updated.insert(8 , 'ch_ra' , ch_ra)
source_updated.insert(9 , 'ch_dec' , ch_dec)
source_updated.insert(10 , 'xoffset' , offset)
source_updated.insert(11 , 'num_obs_used' , n_obs_list)
display(source_updated)
#display(df_combined)
df_combined = df_combined.reset_index(drop=True)
df_combined = df_combined.set_index('obs_id')
df_combined

100%|██████████| 314/314 [00:29<00:00, 10.70it/s]


,Name,Disc. Date,Mag.,R.A.,Dec.,num_obs,Type,chandra_name,ch_ra,ch_dec,xoffset,num_obs_used
0,CSS120424:125906+242634,2012/04/24,NaN,12:59:05.78,+24:26:34.4,572,Candidate_CV,2CXO J004247.1+411619,10.696540,41.27215,0.157954,25
1,W32,NaN,NaN,00:24:05.716,-72:04:49.44,556,CataclyV,2CXO J002405.6-720449,6.023632,-72.08036,0.259094,12
2,CXOGlb_J002405.2-720446,NaN,NaN,00:24:05.216,-72:04:46.59,523,CataclyV,2CXO J002405.2-720446,6.021793,-72.07968,0.268794,8
3,Cl__NGC_104_MAM_M54898,NaN,NaN,00:24:06.013,-72:04:56.20,523,CataclyV,2CXO J002406.0-720456,6.025117,-72.08228,0.071556,13
4,CXOGlb_J002404.9-720455,NaN,NaN,00:24:04.913,-72:04:55.36,478,CataclyV,2CXO J002404.9-720455,6.020589,-72.08210,0.249207,13
...,...,...,...,...,...,...,...,...,...,...,...,...
309,N2403_2008-01a,NaN,NaN,07:37:01.33,+65:35:14.7,1,Candidate_Nova,2CXO J174311.1-271622,265.796300,-27.27295,0.877141,1
310,MASTER_OT_JJ040556.85+714813.1,2013/Dec/23,18.2,04:05:56.85,+71:48:13.1,1,Known_CV,2CXO J040556.8+714816,61.487060,71.80466,3.687046,1
311,CX25,NaN,NaN,19:10:51.957,-59:58:40.55,1,Candidate_CV,2CXO J191051.9-595840,287.716500,-59.97791,0.073509,1
312,MASTER_OT_JJ105545.20+573109.7,2014/Mar/04,15.5,10:55:45.20,+57:31:09.7,1,Known_CV,2CXO J105545.1+573108,163.938100,57.51916,0.891817,1


,src_id,num_obs,sepn,instrument,detect_stack_id,name,ra,dec,obsid,obi,...,var_inter_sigma_s,var_inter_index_u,var_inter_prob_u,var_inter_sigma_u,acis_num,acis_hetg_num,acis_letg_num,hrc_num,hrc_hetg_num,hrc_letg_num
obs_id,,,,,,,,,,,,,,,,,,,,,
CV_0_obs_0,CV_0,25,0.157954,ACIS,acisfJ0042460p411550_001,2CXO J004247.1+411619,10.69654,41.27215,10553,0,...,2.546e-05,NaN,NaN,NaN,109,0,0,63,0,0
CV_0_obs_1,CV_0,25,0.157954,ACIS,acisfJ0042460p411550_001,2CXO J004247.1+411619,10.69654,41.27215,14931,0,...,2.546e-05,NaN,NaN,NaN,109,0,0,63,0,0
CV_0_obs_2,CV_0,25,0.157954,ACIS,acisfJ0042460p411550_001,2CXO J004247.1+411619,10.69654,41.27215,14930,0,...,2.546e-05,NaN,NaN,NaN,109,0,0,63,0,0
CV_0_obs_3,CV_0,25,0.157954,ACIS,acisfJ0042460p411550_001,2CXO J004247.1+411619,10.69654,41.27215,14929,0,...,2.546e-05,NaN,NaN,NaN,109,0,0,63,0,0
CV_0_obs_4,CV_0,25,0.157954,ACIS,acisfJ0042460p411550_001,2CXO J004247.1+411619,10.69654,41.27215,14928,0,...,2.546e-05,NaN,NaN,NaN,109,0,0,63,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CV_309_obs_0,CV_309,1,0.877141,ACIS,acisfJ1743317m271131_001,2CXO J174311.1-271622,265.79630,-27.27295,8658,1,...,NaN,NaN,NaN,NaN,2,0,0,0,0,0
CV_310_obs_0,CV_310,1,3.687046,ACIS,acisfJ0403173p714305_001,2CXO J040556.8+714816,61.48706,71.80466,15999,0,...,NaN,NaN,NaN,NaN,1,0,0,0,0,0
CV_311_obs_0,CV_311,1,0.073509,ACIS,acisfJ1910494m595835_001,2CXO J191051.9-595840,287.71650,-59.97791,6612,0,...,1.036e-07,NaN,NaN,NaN,2,0,0,0,0,0


## Save raw table

In [65]:
df_combined.replace(np.nan , '')
df_combined.to_csv('cv_raw_data.csv' ,)

## Filter flags

In [66]:
col_to_drop = [
    'instrument',
 'detect_stack_id',
 'obsid',
 'obi',
 'src_rate_aper_m',
 'src_rate_aper_lolim_m',
 'src_rate_aper_hilim_m',
 'src_cnts_aper_u',
 'src_cnts_aper_lolim_u',
 'src_cnts_aper_hilim_u',
 'src_cnts_aper_h',
 'src_cnts_aper_lolim_h',
 'src_cnts_aper_hilim_h',
 'sat_src_flag',
 'src_rate_aper_s',
 'src_rate_aper_lolim_s',
 'src_rate_aper_hilim_s',
 'gti_start',
 'gti_stop',
 'gti_obs',
 'gti_end',
 'gti_mjd_obs',
 'mjd_ref',
 'src_rate_aper_b',
 'src_rate_aper_lolim_b',
 'src_rate_aper_hilim_b',
 'theta',
 'phi',
 'region_id',
 'src_cnts_aper_m',
 'src_cnts_aper_lolim_m',
 'src_cnts_aper_hilim_m',
 'src_cnts_aper_s',
 'src_cnts_aper_lolim_s',
 'src_cnts_aper_hilim_s',
 'src_rate_aper_h',
 'src_rate_aper_lolim_h',
 'src_rate_aper_u',
 'src_rate_aper_lolim_u',
 'src_rate_aper_hilim_u',
 'src_cnts_aper_b',
 'src_cnts_aper_lolim_b',
 'src_cnts_aper_hilim_b',
 'apec_kt',
 'apec_kt_lolim',
 'apec_kt_hilim',
 'apec_kt_rhat',
 'apec_abund',
 'apec_abund_lolim',
 'apec_abund_hilim',
 'apec_abund_rhat',
 'apec_z',
 'apec_z_lolim',
 'apec_z_hilim',
 'apec_z_rhat',
 'apec_nh',
 'apec_nh_lolim',
 'apec_nh_hilim',
 'apec_nh_rhat',
 'apec_norm',
 'apec_norm_lolim',
 'apec_norm_hilim',
 'apec_norm_rhat',
 'apec_stat',
 'detector',
 'match_type',
 'cnts_aper_b',
 'cnts_aperbkg_b',
 'cnts_aper_h',
 'cnts_aperbkg_h',
 'cnts_aper_m',
 'cnts_aperbkg_m',
 'cnts_aper_s',
 'cnts_aperbkg_s',
 'cnts_aper_u',
 'cnts_aperbkg_u',
 'pileup_flag',
 'mstr_sat_src_flag',
 'mstr_streak_src_flag',
 'mstr_flux_aper_b',
 'mstr_flux_aper_lolim_b',
 'mstr_flux_aper_hilim_b',
 'acis_num',
 'acis_hetg_num',
 'acis_letg_num',
 'hrc_num',
 'hrc_hetg_num',
 'hrc_letg_num' , 
 'streak_src_flag',
 'src_rate_aper_hilim_h',
]

In [67]:
df = pd.read_csv('cv_raw_data.csv' , index_col='obs_id')
filter_flags = [
    'pileup_flag',
    'mstr_sat_src_flag',
    'mstr_streak_src_flag',
    'sat_src_flag',
    'streak_src_flag',
    ]
df = df.replace({
    '  NAN' : np.nan , 
    '      NaN' : np.nan , 
    'FALSE' : False , 
    'False' : False , 
    ' TRUE' : True , 
    
    })
print('BEFORE FILTERING')
display(df)
for f in filter_flags:
    df = df[df[f]==False]
print('AFTER FILTERING')
display(df)
df = df.drop(columns=col_to_drop)
df

/home/kumaran/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (27,230) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


BEFORE FILTERING


,src_id,num_obs,sepn,instrument,detect_stack_id,name,ra,dec,obsid,obi,...,var_inter_sigma_s,var_inter_index_u,var_inter_prob_u,var_inter_sigma_u,acis_num,acis_hetg_num,acis_letg_num,hrc_num,hrc_hetg_num,hrc_letg_num
obs_id,,,,,,,,,,,,,,,,,,,,,
CV_0_obs_0,CV_0,25,0.157954,ACIS,acisfJ0042460p411550_001,2CXO J004247.1+411619,10.69654,41.27215,10553,0,...,2.546e-05,NaN,NaN,NaN,109,0,0,63,0,0
CV_0_obs_1,CV_0,25,0.157954,ACIS,acisfJ0042460p411550_001,2CXO J004247.1+411619,10.69654,41.27215,14931,0,...,2.546e-05,NaN,NaN,NaN,109,0,0,63,0,0
CV_0_obs_2,CV_0,25,0.157954,ACIS,acisfJ0042460p411550_001,2CXO J004247.1+411619,10.69654,41.27215,14930,0,...,2.546e-05,NaN,NaN,NaN,109,0,0,63,0,0
CV_0_obs_3,CV_0,25,0.157954,ACIS,acisfJ0042460p411550_001,2CXO J004247.1+411619,10.69654,41.27215,14929,0,...,2.546e-05,NaN,NaN,NaN,109,0,0,63,0,0
CV_0_obs_4,CV_0,25,0.157954,ACIS,acisfJ0042460p411550_001,2CXO J004247.1+411619,10.69654,41.27215,14928,0,...,2.546e-05,NaN,NaN,NaN,109,0,0,63,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CV_309_obs_0,CV_309,1,0.877141,ACIS,acisfJ1743317m271131_001,2CXO J174311.1-271622,265.79630,-27.27295,8658,1,...,NaN,NaN,NaN,NaN,2,0,0,0,0,0
CV_310_obs_0,CV_310,1,3.687046,ACIS,acisfJ0403173p714305_001,2CXO J040556.8+714816,61.48706,71.80466,15999,0,...,NaN,NaN,NaN,NaN,1,0,0,0,0,0
CV_311_obs_0,CV_311,1,0.073509,ACIS,acisfJ1910494m595835_001,2CXO J191051.9-595840,287.71650,-59.97791,6612,0,...,1.0359999999999999e-07,NaN,NaN,NaN,2,0,0,0,0,0


AFTER FILTERING


,src_id,num_obs,sepn,instrument,detect_stack_id,name,ra,dec,obsid,obi,...,var_inter_sigma_s,var_inter_index_u,var_inter_prob_u,var_inter_sigma_u,acis_num,acis_hetg_num,acis_letg_num,hrc_num,hrc_hetg_num,hrc_letg_num
obs_id,,,,,,,,,,,,,,,,,,,,,
CV_0_obs_0,CV_0,25,0.157954,ACIS,acisfJ0042460p411550_001,2CXO J004247.1+411619,10.69654,41.27215,10553,0,...,2.546e-05,NaN,NaN,NaN,109,0,0,63,0,0
CV_0_obs_1,CV_0,25,0.157954,ACIS,acisfJ0042460p411550_001,2CXO J004247.1+411619,10.69654,41.27215,14931,0,...,2.546e-05,NaN,NaN,NaN,109,0,0,63,0,0
CV_0_obs_2,CV_0,25,0.157954,ACIS,acisfJ0042460p411550_001,2CXO J004247.1+411619,10.69654,41.27215,14930,0,...,2.546e-05,NaN,NaN,NaN,109,0,0,63,0,0
CV_0_obs_3,CV_0,25,0.157954,ACIS,acisfJ0042460p411550_001,2CXO J004247.1+411619,10.69654,41.27215,14929,0,...,2.546e-05,NaN,NaN,NaN,109,0,0,63,0,0
CV_0_obs_4,CV_0,25,0.157954,ACIS,acisfJ0042460p411550_001,2CXO J004247.1+411619,10.69654,41.27215,14928,0,...,2.546e-05,NaN,NaN,NaN,109,0,0,63,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CV_309_obs_0,CV_309,1,0.877141,ACIS,acisfJ1743317m271131_001,2CXO J174311.1-271622,265.79630,-27.27295,8658,1,...,NaN,NaN,NaN,NaN,2,0,0,0,0,0
CV_310_obs_0,CV_310,1,3.687046,ACIS,acisfJ0403173p714305_001,2CXO J040556.8+714816,61.48706,71.80466,15999,0,...,NaN,NaN,NaN,NaN,1,0,0,0,0,0
CV_311_obs_0,CV_311,1,0.073509,ACIS,acisfJ1910494m595835_001,2CXO J191051.9-595840,287.71650,-59.97791,6612,0,...,1.0359999999999999e-07,NaN,NaN,NaN,2,0,0,0,0,0


,src_id,num_obs,sepn,name,ra,dec,photflux_aper_s,photflux_aper_lolim_s,photflux_aper_hilim_s,flux_aper_s,...,var_inter_sigma_h,var_inter_index_m,var_inter_prob_m,var_inter_sigma_m,var_inter_index_s,var_inter_prob_s,var_inter_sigma_s,var_inter_index_u,var_inter_prob_u,var_inter_sigma_u
obs_id,,,,,,,,,,,,,,,,,,,,,
CV_0_obs_0,CV_0,25,0.157954,2CXO J004247.1+411619,10.69654,41.27215,0.000e+00,0.000e+00,2.001e-06,0.000e+00,...,NaN,NaN,NaN,NaN,5.0,0.989,2.546e-05,NaN,NaN,NaN
CV_0_obs_1,CV_0,25,0.157954,2CXO J004247.1+411619,10.69654,41.27215,0.000e+00,0.000e+00,3.662e-06,0.000e+00,...,NaN,NaN,NaN,NaN,5.0,0.989,2.546e-05,NaN,NaN,NaN
CV_0_obs_2,CV_0,25,0.157954,2CXO J004247.1+411619,10.69654,41.27215,0.000e+00,0.000e+00,2.274e-06,0.000e+00,...,NaN,NaN,NaN,NaN,5.0,0.989,2.546e-05,NaN,NaN,NaN
CV_0_obs_3,CV_0,25,0.157954,2CXO J004247.1+411619,10.69654,41.27215,1.549e-06,0.000e+00,3.098e-06,3.096e-15,...,NaN,NaN,NaN,NaN,5.0,0.989,2.546e-05,NaN,NaN,NaN
CV_0_obs_4,CV_0,25,0.157954,2CXO J004247.1+411619,10.69654,41.27215,0.000e+00,0.000e+00,1.748e-06,0.000e+00,...,NaN,NaN,NaN,NaN,5.0,0.989,2.546e-05,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CV_309_obs_0,CV_309,1,0.877141,2CXO J174311.1-271622,265.79630,-27.27295,0.0,0.0,3.235e-06,0.0,...,4.404e-06,5.0,0.802,1.246e-06,NaN,NaN,NaN,NaN,NaN,NaN
CV_310_obs_0,CV_310,1,3.687046,2CXO J040556.8+714816,61.48706,71.80466,2.496e-06,1.182e-06,3.809e-06,1.63e-15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CV_311_obs_0,CV_311,1,0.073509,2CXO J191051.9-595840,287.71650,-59.97791,1.8730000000000002e-07,6.688e-08,3.077e-07,3.668e-16,...,NaN,6.0,0.983,1.8219999999999998e-07,5.0,0.774,1.0359999999999999e-07,NaN,NaN,NaN


## Check data types 

In [68]:
arranged_cols = [
    "src_id",
    "num_obs",
    "sepn",
    "name",
    "ra",
    "dec",
    "livetime",
    "significance",
    "likelihood",
    "flux_significance_b",
    "flux_significance_h",
    "flux_significance_m",
    "flux_significance_s",
    "flux_significance_u",
    "photflux_aper_s",
    "photflux_aper_lolim_s",
    "photflux_aper_hilim_s",
    "flux_aper_s",
    "flux_aper_lolim_s",
    "flux_aper_hilim_s",
    "photflux_aper_b",
    "photflux_aper_lolim_b",
    "photflux_aper_hilim_b",
    "flux_aper_h",
    "flux_aper_lolim_h",
    "flux_aper_hilim_h",
    "photflux_aper_u",
    "photflux_aper_lolim_u",
    "photflux_aper_hilim_u",
    "flux_aper_b",
    "flux_aper_lolim_b",
    "flux_aper_hilim_b",
    "flux_aper_u",
    "flux_aper_lolim_u",
    "flux_aper_hilim_u",
    "flux_aper_m",
    "flux_aper_lolim_m",
    "flux_aper_hilim_m",
    "photflux_aper_h",
    "photflux_aper_lolim_h",
    "photflux_aper_hilim_h",
    "photflux_aper_m",
    "photflux_aper_lolim_m",
    "photflux_aper_hilim_m",
    "hard_hm",
    "hard_hm_lolim",
    "hard_hm_hilim",
    "hard_hs",
    "hard_hs_lolim",
    "hard_hs_hilim",
    "hard_ms",
    "hard_ms_lolim",
    "hard_ms_hilim",
    "flux_powlaw",
    "flux_powlaw_lolim",
    "flux_powlaw_hilim",
    "powlaw_gamma",
    "powlaw_gamma_lolim",
    "powlaw_gamma_hilim",
    "powlaw_gamma_rhat",
    "powlaw_nh",
    "powlaw_nh_lolim",
    "powlaw_nh_hilim",
    "powlaw_nh_rhat",
    "powlaw_ampl",
    "powlaw_ampl_lolim",
    "powlaw_ampl_hilim",
    "powlaw_ampl_rhat",
    "powlaw_stat",
    "flux_bb",
    "flux_bb_lolim",
    "flux_bb_hilim",
    "bb_kt",
    "bb_kt_lolim",
    "bb_kt_hilim",
    "bb_kt_rhat",
    "bb_nh",
    "bb_nh_lolim",
    "bb_nh_hilim",
    "bb_nh_rhat",
    "bb_ampl",
    "bb_ampl_lolim",
    "bb_ampl_hilim",
    "bb_ampl_rhat",
    "bb_stat",
    "flux_brems",
    "flux_brems_lolim",
    "flux_brems_hilim",
    "brems_kt",
    "brems_kt_lolim",
    "brems_kt_hilim",
    "brems_kt_rhat",
    "brems_nh",
    "brems_nh_lolim",
    "brems_nh_hilim",
    "brems_nh_rhat",
    "brems_norm",
    "brems_norm_lolim",
    "brems_norm_hilim",
    "brems_norm_rhat",
    "brems_stat",
    "flux_apec",
    "flux_apec_lolim",
    "flux_apec_hilim",
    "var_index_b",
    "var_prob_b",
    "ks_prob_b",
    "kp_prob_b",
    "var_sigma_b",
    "var_mean_b",
    "var_min_b",
    "var_max_b",
    "var_index_h",
    "var_prob_h",
    "ks_prob_h",
    "kp_prob_h",
    "var_sigma_h",
    "var_mean_h",
    "var_min_h",
    "var_max_h",
    "var_index_m",
    "var_prob_m",
    "ks_prob_m",
    "kp_prob_m",
    "var_sigma_m",
    "var_mean_m",
    "var_min_m",
    "var_max_m",
    "var_index_s",
    "var_prob_s",
    "ks_prob_s",
    "kp_prob_s",
    "var_sigma_s",
    "var_mean_s",
    "var_min_s",
    "var_max_s",
    "var_index_u",
    "var_prob_u",
    "ks_prob_u",
    "kp_prob_u",
    "var_sigma_u",
    "var_mean_u",
    "var_min_u",
    "var_max_u",
    "var_inter_index_b",
    "var_inter_prob_b",
    "var_inter_sigma_b",
    "var_inter_index_h",
    "var_inter_prob_h",
    "var_inter_sigma_h",
    "var_inter_index_m",
    "var_inter_prob_m",
    "var_inter_sigma_m",
    "var_inter_index_s",
    "var_inter_prob_s",
    "var_inter_sigma_s",
    "var_inter_index_u",
    "var_inter_prob_u",
    "var_inter_sigma_u",
]

id_cols = [
   "src_id",
    "num_obs",
    "sepn",
    "name",
    "ra",
    "dec",
    "livetime",
    "significance",
    "likelihood",
]
val_cols = [
    
]

In [69]:
df_id = df[id_cols]
df_val = df.drop(columns=id_cols)
val_col_list = df_val.columns.to_list()
for v in val_col_list:
    df_val.loc[:,v] = pd.to_numeric(df_val.loc[:,v])
df_val

,photflux_aper_s,photflux_aper_lolim_s,photflux_aper_hilim_s,flux_aper_s,flux_aper_lolim_s,flux_aper_hilim_s,likelihood_b,likelihood_h,likelihood_m,likelihood_s,...,var_inter_sigma_h,var_inter_index_m,var_inter_prob_m,var_inter_sigma_m,var_inter_index_s,var_inter_prob_s,var_inter_sigma_s,var_inter_index_u,var_inter_prob_u,var_inter_sigma_u
obs_id,,,,,,,,,,,,,,,,,,,,,
CV_0_obs_0,0.000000e+00,0.000000e+00,2.001000e-06,0.000000e+00,0.000000e+00,2.726000e-15,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5.0,0.989,2.546000e-05,NaN,NaN,NaN
CV_0_obs_1,0.000000e+00,0.000000e+00,3.662000e-06,0.000000e+00,0.000000e+00,5.441000e-15,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5.0,0.989,2.546000e-05,NaN,NaN,NaN
CV_0_obs_2,0.000000e+00,0.000000e+00,2.274000e-06,0.000000e+00,0.000000e+00,2.977000e-15,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5.0,0.989,2.546000e-05,NaN,NaN,NaN
CV_0_obs_3,1.549000e-06,0.000000e+00,3.098000e-06,3.096000e-15,0.000000e+00,6.028000e-15,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5.0,0.989,2.546000e-05,NaN,NaN,NaN
CV_0_obs_4,0.000000e+00,0.000000e+00,1.748000e-06,0.000000e+00,0.000000e+00,2.380000e-15,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5.0,0.989,2.546000e-05,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CV_309_obs_0,0.000000e+00,0.000000e+00,3.235000e-06,0.000000e+00,0.000000e+00,4.406000e-15,20.89,10.64,NaN,NaN,...,0.000004,5.0,0.802,1.246000e-06,NaN,NaN,NaN,NaN,NaN,NaN
CV_310_obs_0,2.496000e-06,1.182000e-06,3.809000e-06,1.630000e-15,0.000000e+00,3.341000e-15,53.30,22.69,30.43,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CV_311_obs_0,1.873000e-07,6.688000e-08,3.077000e-07,3.668000e-16,1.310000e-16,6.025000e-16,22.78,NaN,27.12,NaN,...,NaN,6.0,0.983,1.822000e-07,5.0,0.774,1.036000e-07,NaN,NaN,NaN


In [70]:
df_ok = pd.concat([df_id , df_val] , axis=1)
df_ok

,src_id,num_obs,sepn,name,ra,dec,livetime,significance,likelihood,photflux_aper_s,...,var_inter_sigma_h,var_inter_index_m,var_inter_prob_m,var_inter_sigma_m,var_inter_index_s,var_inter_prob_s,var_inter_sigma_s,var_inter_index_u,var_inter_prob_u,var_inter_sigma_u
obs_id,,,,,,,,,,,,,,,,,,,,,
CV_0_obs_0,CV_0,25,0.157954,2CXO J004247.1+411619,10.69654,41.27215,4103.7,52.93,16618.73000,0.000000e+00,...,NaN,NaN,NaN,NaN,5.0,0.989,2.546000e-05,NaN,NaN,NaN
CV_0_obs_1,CV_0,25,0.157954,2CXO J004247.1+411619,10.69654,41.27215,3907.5,52.93,16618.73000,0.000000e+00,...,NaN,NaN,NaN,NaN,5.0,0.989,2.546000e-05,NaN,NaN,NaN
CV_0_obs_2,CV_0,25,0.157954,2CXO J004247.1+411619,10.69654,41.27215,3907.5,52.93,16618.73000,0.000000e+00,...,NaN,NaN,NaN,NaN,5.0,0.989,2.546000e-05,NaN,NaN,NaN
CV_0_obs_3,CV_0,25,0.157954,2CXO J004247.1+411619,10.69654,41.27215,3904.6,52.93,16618.73000,1.549000e-06,...,NaN,NaN,NaN,NaN,5.0,0.989,2.546000e-05,NaN,NaN,NaN
CV_0_obs_4,CV_0,25,0.157954,2CXO J004247.1+411619,10.69654,41.27215,3907.5,52.93,16618.73000,0.000000e+00,...,NaN,NaN,NaN,NaN,5.0,0.989,2.546000e-05,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CV_309_obs_0,CV_309,1,0.877141,2CXO J174311.1-271622,265.79630,-27.27295,2156.9,2.53,20.88602,0.000000e+00,...,0.000004,5.0,0.802,1.246000e-06,NaN,NaN,NaN,NaN,NaN,NaN
CV_310_obs_0,CV_310,1,3.687046,2CXO J040556.8+714816,61.48706,71.80466,28678.8,4.86,53.30116,2.496000e-06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CV_311_obs_0,CV_311,1,0.073509,2CXO J191051.9-595840,287.71650,-59.97791,37966.9,2.61,27.11662,1.873000e-07,...,NaN,6.0,0.983,1.822000e-07,5.0,0.774,1.036000e-07,NaN,NaN,NaN


In [71]:
df_ok.to_csv('cv_data.csv')